In [1]:
import pandas as pd
import unicodedata
import re
from difflib import get_close_matches

In [2]:
stats_2024=pd.read_csv('../../scrapers/stats_scraper/2024_batting_stats.csv')
batter_team_2024=pd.read_csv('batters_team_2024.csv')

In [3]:
stats_2024

,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,away_OBP,...,home_AB,home_R,home_H,home_RBI,home_HR,home_BB,home_K,home_AVG,home_OBP,home_SLG
0,F. LindorSS,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.291,0.359,...,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.255,0.354,0.540
1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,1.0,1.0,0.255,0.390,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.407,0.493,0.762
2,P. Alonso1B,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.304,0.411,...,3.0,1.0,1.0,1.0,1.0,0.0,1.0,0.243,0.308,0.421
3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.230,0.291,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.343,0.391,0.497
4,M. VientosDH,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.242,0.306,...,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.246,0.340,0.431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14260,M. Garcia3B,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.231,0.281,...,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.194,0.280,0.353
14261,K. IsbelCF,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0.229,0.287,...,4.0,1.0,2.0,1.0,1.0,0.0,1.0,0.264,0.281,0.432
14262,team,30.0,4.0,5.0,4.0,2.0,3.0,14.0,NaN,NaN,...,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.220,0.272,0.357
14263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.235,0.301,0.437


In [4]:
#normalizing the names, getting rid of accents
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [5]:
batter_team_2024['player'] = batter_team_2024['player'].apply(normalize_name)

In [6]:
#getting rid of the # and the * from the names
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

In [7]:
batter_team_2024['player'] = batter_team_2024['player'].apply(clean_name_symbols)

In [8]:
multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
filtered_df_2 = batter_team_2024[~batter_team_2024['team'].isin(multi_team_flags)].copy()
batter_team_2024 = filtered_df_2.drop_duplicates(subset='player', keep='last').reset_index(drop=True)

In [9]:
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # 1) initial-style (one or more letter-dot sequences, space, surname+optional code)
    m = re.search(r"((?:[A-Za-z]\.)+\s[A-Za-z'-]+)", raw)
    if m:
        name = m.group(1)
        # strip any trailing digits/uppercase letters/hyphens
        strip = re.match(r"^((?:[A-Za-z]\.)+\s[A-Za-z'-]+?)(?:[0-9A-Z-]+)$", name)
        return strip.group(1) if strip else name

    # 2) plain First Last but only up to just before any digit or hyphen
    m2 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)(?=[0-9-])", raw)
    if m2:
        return m2.group(1)

    # 3) fallback to a simple First Last
    m3 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)", raw)
    if m3:
        return m3.group(1)

    return None

stats_2024['short_name'] = stats_2024['away_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2024['player'].tolist()
stats_2024['matched_name'] = stats_2024['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df = stats_2024.merge(batter_team_2024, left_on='matched_name', right_on='player', how='left')

In [10]:
#dropping the collumns we used to match to then match the other column
merged_df = merged_df.drop(columns=['short_name', 'matched_name'])

In [11]:
merged_df.rename(columns={'player': 'away_player', 'team':'away_team'}, inplace=True)

In [12]:
merged_df

,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,away_OBP,...,home_H,home_RBI,home_HR,home_BB,home_K,home_AVG,home_OBP,home_SLG,away_player,away_team
0,F. LindorSS,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.291,0.359,...,1.0,0.0,0.0,0.0,0.0,0.255,0.354,0.540,Francisco Lindor,NYM
1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,1.0,1.0,0.255,0.390,...,0.0,0.0,0.0,0.0,1.0,0.407,0.493,0.762,Juan Soto,NYY
2,P. Alonso1B,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.304,0.411,...,1.0,1.0,1.0,0.0,1.0,0.243,0.308,0.421,Pete Alonso,NYM
3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.230,0.291,...,0.0,0.0,0.0,0.0,1.0,0.343,0.391,0.497,Brandon Nimmo,NYM
4,M. VientosDH,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.242,0.306,...,2.0,0.0,0.0,0.0,0.0,0.246,0.340,0.431,Mark Vientos,NYM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14260,M. Garcia3B,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.231,0.281,...,0.0,0.0,0.0,0.0,1.0,0.194,0.280,0.353,Maikel Garcia,KCR
14261,K. IsbelCF,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0.229,0.287,...,2.0,1.0,1.0,0.0,1.0,0.264,0.281,0.432,Kyle Isbel,KCR
14262,team,30.0,4.0,5.0,4.0,2.0,3.0,14.0,NaN,NaN,...,1.0,0.0,0.0,1.0,0.0,0.220,0.272,0.357,NaN,NaN
14263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.235,0.301,0.437,NaN,NaN


In [13]:
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # 1) initial-style (one or more letter-dot sequences, space, surname+optional code)
    m = re.search(r"((?:[A-Za-z]\.)+\s[A-Za-z'-]+)", raw)
    if m:
        name = m.group(1)
        # strip any trailing digits/uppercase letters/hyphens
        strip = re.match(r"^((?:[A-Za-z]\.)+\s[A-Za-z'-]+?)(?:[0-9A-Z-]+)$", name)
        return strip.group(1) if strip else name

    # 2) plain First Last but only up to just before any digit or hyphen
    m2 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)(?=[0-9-])", raw)
    if m2:
        return m2.group(1)

    # 3) fallback to a simple First Last
    m3 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)", raw)
    if m3:
        return m3.group(1)

    return None

merged_df['short_name'] = merged_df['home_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2024['player'].tolist()
merged_df['matched_name'] = merged_df['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_5 = merged_df.merge(batter_team_2024, left_on='matched_name', right_on='player', how='left')

In [14]:
#dropping the collumns we used to match to then match the other column
merged_df_5 = merged_df_5.drop(columns=['short_name', 'matched_name'])

In [15]:
merged_df_5.rename(columns={'player': 'home_player', 'team':'home_team'}, inplace=True)

In [16]:
import pandas as pd

def assign_game_ids_by_scan(df: pd.DataFrame,
                            away_col: str    = 'away_hitters',
                            home_col: str    = 'home_hitters',
                            summary_val: str = 'team',
                            id_col: str      = 'gameID') -> pd.DataFrame:
    """
    Walk down the DataFrame row by row.  
    - Start gameID = 1.  
    - At each row: assign current gameID.  
    - If row[away_col] == summary_val, mark away_seen.  
      If row[home_col] == summary_val, mark home_seen.  
    - As soon as both away_seen & home_seen are True, increment gameID, reset both flags.
    """
    df = df.copy()
    game_id = 1
    away_seen = False
    home_seen = False
    ids = []

    for _, row in df.iterrows():
        # assign the current game_id
        ids.append(game_id)

        # check for each summary marker
        if str(row[away_col]).strip().lower() == summary_val:
            away_seen = True
        if str(row[home_col]).strip().lower() == summary_val:
            home_seen = True

        # once both sides have had their 'team' row, we end the block
        if away_seen and home_seen:
            game_id  += 1
            away_seen = False
            home_seen = False

    df[id_col] = ids
    return df

# Apply it to your merged_df_5:
merged_df_5 = assign_game_ids_by_scan(
    merged_df_5,
    away_col='away_hitters',
    home_col='home_hitters',
    summary_val='team',
    id_col='gameID'
)

# Quick check
print(merged_df_5[['away_hitters','home_hitters','gameID']].iloc[150:180])


         away_hitters         home_hitters  gameID
150        C. Keith2B         J. MirandaDH      14
151       J. MalloyDH       A. MartinPR-DH      14
152       A. BaddooLF         C. Santana1B      14
153        J. RogersC          B. BuxtonCF      14
154     R. KreidlerSS          R. JeffersC      14
155              team       W. CastroLF-2B      14
156               NaN       K. Farmer2B-3B      14
157               NaN                 team      14
158        B. Stott2B         N. Hoerner2B      15
159       T. TurnerSS           M. Busch1B      15
160         A. Bohm3B       C. BellingerRF      15
161        B. MarshLF          S. SuzukiDH      15
162  N. CastellanosRF            I. HappLF      15
163      K. Clemens1B           C. Morel3B      15
164         D. DahlDH         D. SwansonSS      15
165        G. StubbsC  P. Crow-ArmstrongCF      15
166        J. RojasCF             T. NidoC      15
167              team    aM. MastrobuoniPH      15
168               NaN          

In [17]:
for col in ['home_team','away_team']:
    merged_df_5[col] = (
        merged_df_5
        .groupby('gameID')[col]
        .transform(lambda x: x.ffill().bfill())
    )

In [18]:
merged_df_5.to_csv('2024_batting_stats_edited.csv')